In [1]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

2023-06-14 10:29:34.298838: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 10:29:35.151187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)

In [3]:
env.reset()

array([ 0.01533464, -0.01066766, -0.00341502, -0.03386957], dtype=float32)

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
     
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() 

Episode:1 Score:11.0
Episode:2 Score:33.0
Episode:3 Score:16.0
Episode:4 Score:21.0
Episode:5 Score:37.0


In [5]:
log_path=os.path.join("Training","Logs")

In [6]:
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log=log_path)

Using cuda device


In [7]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 660  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 695         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008312503 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00124    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.18        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0193     |
|    value_loss           | 59.4        |
-----------------------------------------
---

In [8]:
PPO_Path = os.path.join("Training","Saved Models","PPO_Model_Cartpole")

In [9]:
model.save(PPO_Path)

/home/shreyas/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [10]:
del model

In [11]:
model = PPO.load(PPO_Path)

In [12]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

env.close()

/home/shreyas/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [13]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
     
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close() 

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


In [14]:
training_log_path = os.path.join(log_path, 'PPO_8')

In [15]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [16]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [17]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [18]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=5000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [19]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cuda device


In [20]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 1004 |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 835         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008848423 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00468    |
|    learning_rate        | 0.0003      |
|    loss                 | 10.7        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0184     |
|    value_loss           | 63.1        |
-----------------------------------------
Eva

In [21]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [22]:
env.close()

In [23]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [24]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cuda device


/home/shreyas/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/policies.py:457: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [25]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 906  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 706         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014916702 |
|    clip_fraction        | 0.204       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00543    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.96        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0251     |
|    value_loss           | 20.2        |
-----------------------------------------
Eval num_timesteps=5000, episode_r

In [26]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [27]:
env.close()

In [28]:
from stable_baselines3 import DQN

In [29]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cuda device


In [30]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.963    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3420     |
|    time_elapsed     | 0        |
|    total_timesteps  | 78       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.934    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3978     |
|    time_elapsed     | 0        |
|    total_timesteps  | 139      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.898    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4533     |
|    time_elapsed     | 0        |
|    total_timesteps  | 214      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 6430     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2272     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 6437     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2359     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 6424     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2415     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 208      |
|    fps              | 6081     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4708     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 212      |
|    fps              | 6114     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4851     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 6114     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4919     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 316      |
|    fps              | 6445     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7334     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 320      |
|    fps              | 6449     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7403     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 6453     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7467     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 424      |
|    fps              | 6547     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9425     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 428      |
|    fps              | 6548     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9481     |
----------------------------------
Eval num_timesteps=9520, episode_reward=29.60 +/- 4.54
Episode length: 29.60 +/- 4.54
----------------------------------
| eval/               |          |
|    mean_ep_length   | 29.6     |
|    mean_reward      | 29.6     |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 9520     |
----

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 524      |
|    fps              | 6184     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11591    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 528      |
|    fps              | 6192     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11680    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 6198     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11772    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 632      |
|    fps              | 6350     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14214    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 636      |
|    fps              | 6352     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14270    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 6355     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14335    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 732      |
|    fps              | 6173     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16489    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 736      |
|    fps              | 6176     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16570    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 740      |
|    fps              | 6187     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16707    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 840      |
|    fps              | 6243     |
|    time_elapsed     | 3        |
|    total_timesteps  | 18927    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 844      |
|    fps              | 6240     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19060    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 848      |
|    fps              | 6243     |
|    time_elapsed     | 3        |
|    total_timesteps  | 19150    |
----------------------------------
----------------------------------
| rollout/          

In [31]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [32]:
model.save(dqn_path)

In [33]:
model = DQN.load(dqn_path, env=env)

In [34]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(37.7, 12.000416659433121)

In [35]:
env.close()